### 1. Import packages

In [1]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")

import gym

### 2. Helper functions

In [2]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    
    return state


### 3. Initialize the environment

In [3]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
    #'fileX_template_path':'./IUAF9901.MZX',
    #'experiment_number':1
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = len(env.observation)+len(env.observation['sw'])-1
nb_actions = 25
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)

Observation: {'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}
27 9

Ram information received from DASSAT will has 35 dimensions.
There are 25 possible actions at each step.
Discrete? False


### 4. Define the network

In [4]:
class QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=128*2,fc2_units=128*2,fc3_units=128*2):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, fc3_units)
        self.fc4 = nn.Linear(fc3_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        y = F.relu(self.fc2(x))
        z = F.relu(self.fc3(y))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return self.fc4(z)

In [6]:
trained_model = QNetwork(35, 5)
trained_model.load_state_dict(torch.load('/home/rant3/focal/model2943.pth'))
#trained_model.load_state_dict(torch.load('/home/rant3/focal/model2400.pth'))

<All keys matched successfully>

5. Define the Replay Buffer

In [7]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
#         action = dict2array(action)
#         print('transfored actions:',action)
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
#         for e in experiences:
# #             print(e)
# #             print(e.state.shape, e.next_state.shape)
#             break
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [8]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 640         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 60                # for update of target network parameters
LR = 1e-5               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [9]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, trained_model):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size
        # Q-Network
        self.qnetwork_local = trained_model
        #self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.numpy()


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
#         print(Q_targets_next)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
#         print('out',self.qnetwork_local(states).shape)
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from40, 0, 0,
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())

In [10]:
def get_reward(state, n_action, w_action, next_state, done, k1, k2, k3, k4, n_amount, w_amount):
    #if done, return Yield (topwt) - k*cost
    penalty1 = 0
    penalty2 = 0
    #k1=10**(-3)*(i_episode*2)
    # base cost is current input action
    if n_amount > 250:
        if n_action!=0:
            penalty1 = (n_amount-250)
    if w_amount > 600:
        if w_action!= 0:
            penalty2=(w_amount-600)
    if done:
        reward = state[29] - k1*n_action - k2*w_action- k3*penalty1-k4*penalty2
        return reward
    #if done, return Yield (topwt) - k*cost
    else:
        reward = -k1*(n_action + 10*state[25]) - k2*w_action- k3*penalty1-k4*penalty2
        return reward
    # otherwise, return -k*(action+leaching)

### DQN for nitrogen management

In [11]:
agent = Agent(state_size=ram_dimensions, action_size=nb_actions, trained_model=trained_model)

In [13]:
env.reset()
Action_list=[]
water_list=[]
Nleach_list = []
Yield_list=[]
reward_list=[]
action_number_list=[]
for i in range(1):
    #print('')
    env.reset()
    n_amount=0
    w_amount=0
    n_list=[]
    w_list=[]
    istage_list=[]
    vstage_list=[]
    sw_list=[]
    #print(env.observation)
    #date = 1
    #print(date)
    while not env.done:
        #print(observation)
        #print(observation)
        # observation_list = env.observation_dict_to_array(observation)
        state=env.observation
        state = dict2array(state)
        action1=agent.act(state,0)
        action_number_list.append(action1)
        i1=(action1%5)*40
        i2=int(action1/5)*6
        #print(action1)
        action = {
                    'anfer': i1,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': i2,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
        n_list.append(action['anfer'])
        w_list.append(action['amir'])
        n_amount += action['anfer']
        w_amount += action['amir']
        istage_list.append(env.observation['istage'])
        vstage_list.append(env.observation['vstage'])
        sw_list.append(env.observation['sw'][8])
        observation, reward, done, info = env.step(action)
        #print('observation is', observation)
        #real_ob = add_fert(observation, action_dict['anfer'])
        #print('real_ob is ', real_ob)
        #env.observation = real_ob
        #print('final ob is:', env.observation)
        #real_s=get_real_state(env._state, action_dict['anfer'])
        #env._state=real_s
        #print('env_state', env._state)
        #print(date)
        #print(reward)
        #print(done)
        if done:
            final_ob=env.observation
            #print(final_ob)
            action=final_ob['cumsumfert']
            Nleach=final_ob['cleach']
            Yield=final_ob['grnwt']
            Action_list.append(action)
            Nleach_list.append(Nleach)
            Yield_list.append(Yield)
            water_list.append(w_amount)
            reward=0.158*Yield-0.79*action-1.1*w_amount-0*Nleach
            reward_list.append(reward)
            #print(env.observation)
            #print(env.history['action'])
            #print(env.history['reward'])
print('')
print('Action is', action_number_list)
print('')
print('N amount is', Action_list)
print('average N input is', np.mean(Action_list))
print('action is', n_list)
print('')
print('water amount is', water_list)
print('averagee water amount is', np.mean(water_list))
print(w_list)
print('')
print('N leaching is', Nleach_list)
print('average Leaching is', np.mean(Nleach_list))
print('')
print('Yield is', Yield_list)
print('average Yield is', np.mean(Yield_list))
print('')
print('Reward is', reward_list)
print('average reward is', np.mean(reward_list))



Action is [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 6, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

N amount is [200.0]
average N input is 200.0
action is [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 40, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
num_1=0
num_5=0
num_6=0
for i in range(len(action_number_list)):
    if action_number_list[i] == 1:
        num_1=num_1+1
    if action_number_list[i] == 5:
        num_5=num_5+1
    if action_number_list[i] == 6:
        num_6=num_6+1
print(num_1,num_5,num_6,len(action_number_list),len(action_number_list)-num_1-num_5-num_6)

1 16 4 155 134


In [144]:
import pandas as pd
a = pd.DataFrame(n_list)
a.to_csv('n_list.csv', index=False, header=False)
b=pd.DataFrame(w_list)
b.to_csv('w_list.csv', index=False, header=False)

In [324]:
water_list=[18, 24, 24, 18, 12, 12, 0, 0, 12, 12, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 6, 6, 6, 6, 6, 12, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [43]:
print(len(action_list))
print(len(water_list))
Action_list=[]
Water_list=[]
Nleach_list = []
Yield_list=[]
reward_list=[]
for i in range(1):
    #print('')
    env.reset()
    n_amount=0
    w_amount=0
    n_list=[]
    w_list=[]
    istage_list=[]
    vstage_list=[]
    sw_list=[]
    date = 0
    #print(env.observation)
    #date = 1
    #print(date)
    while not env.done:
        #print(observation)
        #print(observation)
        # observation_list = env.observation_dict_to_array(observation)
        i1=action_list[date]
        i2=water_list[date]
        #if state[3]>80:
        #    i1=0
        #print(action1)
        action = {
                    'anfer': i1,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': i2,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
        n_list.append(action['anfer'])
        w_list.append(action['amir'])
        n_amount += action['anfer']
        w_amount += action['amir']
        istage_list.append(env.observation['istage'])
        vstage_list.append(env.observation['vstage'])
        sw_list.append(env.observation['sw'][8])
        observation, reward, done, info = env.step(action)
        date=date+1
        #print('observation is', observation)
        #real_ob = add_fert(observation, action_dict['anfer'])
        #print('real_ob is ', real_ob)
        #env.observation = real_ob
        #print('final ob is:', env.observation)
        #real_s=get_real_state(env._state, action_dict['anfer'])
        #env._state=real_s
        #print('env_state', env._state)
        #print(date)
        #print(reward)
        #print(done)
        if done:
            final_ob=env.observation
            #print(final_ob)
            action=final_ob['cumsumfert']
            Nleach=final_ob['cleach']
            Yield=final_ob['grnwt']
            Action_list.append(action)
            Nleach_list.append(Nleach)
            Yield_list.append(Yield)
            Water_list.append(w_amount)
            reward=Yield*0.1-1*Nleach-0.1*n_amount-0.05*w_amount
            reward_list.append(reward)
            #print(env.observation)
            #print(env.history['action'])
            #print(env.history['reward'])
print('')
print('N amount is', Action_list)
print('average N input is', np.mean(Action_list))
print('')
print('water amount is', Water_list)
print('averagee water amount is', np.mean(Water_list))
print('')
print('N leaching is', Nleach_list)
print('average Leaching is', np.mean(Nleach_list))
print('')
print('Yield is', Yield_list)
print('average Yield is', np.mean(Yield_list))
print('')
print('Reward is', reward_list)
print('average reward is', np.mean(reward_list))

155
155

N amount is [240.0]
average N input is 240.0

water amount is [390]
averagee water amount is 390.0

N leaching is [60.12512969970703]
average Leaching is 60.12512969970703

Yield is [11290.998480187554]
average Yield is 11290.998480187554

Reward is [1025.4747183190484]
average reward is 1025.4747183190484
